In [4]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate
from data.configs.demo import config
from torch.utils.data import DataLoader

In [5]:
class fuc:
    def __init__(self):
        pass
        self.a = 1
    
    def b(self):
        pass

a = fuc()
vars(a)


{'a': 1}

In [4]:
fuc.b('c')

cuk
shit


In [5]:
from utils.MIND import MIND_bert

In [7]:
train_path = "C:/MIND/MINDdemo_train/"
config.embedding = 'bert'
config.bert = 'bert-base-uncased'
a = MIND_bert(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [32]:
re.search('/?(.*)?.tsv', 'data/MIND/MINDlarge_train/behaviors.tsv').group(1)

'data/MIND/MINDlarge_train/behaviors'

In [43]:
s = 'a/b/c/xxx.py'
res = re.search('(\w*)\.py', s).group(1)
res

'xxx'

In [9]:
loader_train = DataLoader(a, batch_size=config.batch_size, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, collate_fn=my_collate)

In [10]:
next(iter(loader_train))

{'user_index': tensor([[1],
         [2],
         [3],
         [4],
         [5],
         [6],
         [6],
         [7],
         [8],
         [9]], dtype=torch.int32),
 'cdd_id': tensor([[33900, 50689,     0,     0,     0],
         [32187, 37405, 44370, 37210, 35307],
         [ 5767, 38208, 46822, 46722, 12325],
         [50715, 50689, 50106, 50022,     0],
         [31475, 15572, 37210, 40858, 43870],
         [33788, 31917, 44751, 49384,     0],
         [41551, 31917, 44751, 49384,     0],
         [28461, 48748, 35139, 45728, 38162],
         [33650, 44370, 46191, 33280, 45977],
         [41884, 37034, 43039, 36333,     0]], dtype=torch.int32),
 'his_id': tensor([[ 6893, 10050, 15556, 21467, 26358,  4946, 14071, 13911,  2078,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
    

In [ ]:
from transformers import BertTokenizerFast
tok = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
import torch

In [ ]:
from models.Encoders.CNN import CNN_Encoder
from models.Encoders.FIM import FIM_Encoder

In [ ]:
a = CNN_Encoder()

In [ ]:
from models.Encoders.FIM import FIM_Encoder
import torch
from data.configs.demo import config
config.embedding_dim = 5
config.hidden_dim = 6
a = torch.rand(2,3,4,5)

enc = FIM_Encoder(config)
b = enc(a)

In [ ]:
b[0].size(),b[1].size()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
e = nn.Embedding(2,3)
a = torch.arange(2)
loss = (e(a)**2).sum()
loss.backward()

In [ ]:
e.weight.grad

In [ ]:
e.zero_grad()

In [ ]:
loss = (e.weight**2).sum()
loss.backward()
e.weight.grad